In [37]:
import keras
import numpy as np
from keras.applications import MobileNetV2
from keras.datasets import cifar10
from keras import models
from keras import layers
from keras.utils import to_categorical
import random
from keras.callbacks import EarlyStopping

In [21]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
#take sample
trainindex = random.sample(range(0,len(x_train)),8000)
testindex = random.sample(range(0,len(x_test)),2000)
x_train = x_train[trainindex]
y_train = y_train[trainindex]
x_test = x_test[testindex]
y_test = y_test[testindex]

In [4]:
print(np.unique(y_train))
print(np.unique(y_test))

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


In [22]:
y_train = to_categorical(y_train,num_classes=10)
y_test = to_categorical(y_test,num_classes=10)
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [6]:
def padder(images,width,height):
    padw = int((width-images.shape[1])/2)
    padh = int((height-images.shape[2])/2)
    padded_images = np.pad(images,((0,0),(padw,padw),(padh,padh),(0,0)),'constant')
    return padded_images

x_train = padder(x_train,96,96)
x_test = padder(x_test,96,96)
print(x_train.shape)
print(x_test.shape)

(8000, 96, 96, 3)
(2000, 96, 96, 3)


In [7]:
#Scale Images
x_train = x_train/255
x_test = x_test/255

In [8]:
conv_base = MobileNetV2(weights='imagenet',
                  include_top=False,
                  input_shape=(96, 96, 3))

In [9]:
%%time
x_trainMN = conv_base.predict(x_train)
x_testMN = conv_base.predict(x_test)
x_trainMN = np.reshape(x_trainMN,(x_train.shape[0],3*3*1280))
x_testMN = np.reshape(x_testMN,(x_test.shape[0],3*3*1280))

CPU times: user 19min 52s, sys: 1min 16s, total: 21min 9s
Wall time: 12min 29s


In [10]:
model = models.Sequential()
model.add(layers.Dense(256, input_dim = 3*3*1280, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [12]:
early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0.001, 
                           patience=3, 
                           mode='min', 
                           verbose=1)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_trainMN, y_train, epochs=100, validation_data=(x_testMN,y_test),batch_size=128,callbacks=[early_stop])

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 5s 660us/step - loss: 0.6724 - acc: 0.7738 - val_loss: 1.1567 - val_acc: 0.6595
Epoch 2/100
8000/8000 [==============================] - 4s 551us/step - loss: 0.4386 - acc: 0.8431 - val_loss: 1.0412 - val_acc: 0.6800
Epoch 3/100
8000/8000 [==============================] - 4s 554us/step - loss: 0.3638 - acc: 0.8678 - val_loss: 1.1008 - val_acc: 0.6800
Epoch 4/100
8000/8000 [==============================] - 4s 534us/step - loss: 0.3438 - acc: 0.8768 - val_loss: 1.2213 - val_acc: 0.6690
Epoch 5/100
8000/8000 [==============================] - 4s 541us/step - loss: 0.3191 - acc: 0.8877 - val_loss: 1.3585 - val_acc: 0.6515
Epoch 00005: early stopping


In [ ]:
%reset